In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../../data/train.csv")

In [3]:
df.drop(columns=['id'], inplace=True)
df["outcome"] = df["outcome"].replace({"died": 0, "lived": 1, "euthanized": -1})
df.fillna('empty', inplace=True)
df = pd.get_dummies(df)

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X = df.drop(columns=["outcome"])
y = df["outcome"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y ,random_state=104, test_size=0.25, shuffle=True)

In [7]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [8]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [9]:
def train(model, _X, _y):
    model.fit(_X, _y)    
    return model

In [10]:
def measure(_model, _X, _y):
    
    result = _model.predict(_X)
    
    print(accuracy_score(result, _y))
    print()
    print(confusion_matrix(result, _y, labels=[1, 0, -1]))
    print()
    print(classification_report(result, _y, labels=[1, 0, -1]))

In [11]:
def submit(_model, df_test, _X_train):
    id = df_test['id'].to_list()

    df_test.drop(columns=['id'], inplace=True)
    df_test.fillna('empty', inplace=True)
    df_test = pd.get_dummies(df_test)

    df_test['peristalsis_distend_small'] = False
    df_test['nasogastric_reflux_slight'] = False
    df_test['rectal_exam_feces_serosanguious'] = False

    df_test.rename(columns={'pain_moderate': 'pain_slight'}, inplace=True)

    df_test = df_test[_X_train.columns.to_list()]

    result = _model.predict(df_test)

    df_submission = pd.DataFrame(data={'id': id ,'outcome': list(result)})
    df_submission["outcome"] = df_submission["outcome"].replace({0: "died", 1: "lived", -1: "euthanized"})
    df_submission.to_csv("submission.csv", index = False)

In [36]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
df_test = pd.read_csv("../../data/test.csv")
df_test.head()
parameters = {
    "depth": list(np.arange(10, 100, 10)),
    "learning_rate": list(np.arange(0.01, 1, .1)),
    "iterations": list(np.arange(0, 300, 20))
}
grid = GridSearchCV(estimator=CatBoostClassifier(), param_grid=parameters, cv=2, n_jobs=-1)
model = train(grid, X_train, y_train)
submit(model, df_test, X_train)

c:\GitHub\Health-Outcomes-of-Horses\venv\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
2420 fits failed out of a total of 2700.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "c:\GitHub\Health-Outcomes-of-Horses\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\GitHub\Health-Outcomes-of-Horses\venv\lib\site-packages\catboost\core.py", line 5100, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "c:\GitHub\Health-Outc

0:	learn: 0.9743840	total: 169ms	remaining: 30.2s
1:	learn: 0.8760591	total: 205ms	remaining: 18.3s
2:	learn: 0.7919251	total: 240ms	remaining: 14.1s
3:	learn: 0.7314432	total: 271ms	remaining: 11.9s
4:	learn: 0.6783486	total: 303ms	remaining: 10.6s
5:	learn: 0.6336409	total: 334ms	remaining: 9.68s
6:	learn: 0.5902125	total: 363ms	remaining: 8.96s
7:	learn: 0.5580035	total: 391ms	remaining: 8.4s
8:	learn: 0.5283325	total: 420ms	remaining: 7.98s
9:	learn: 0.4944660	total: 449ms	remaining: 7.62s
10:	learn: 0.4689583	total: 478ms	remaining: 7.34s
11:	learn: 0.4424289	total: 506ms	remaining: 7.09s
12:	learn: 0.4238473	total: 536ms	remaining: 6.88s
13:	learn: 0.4051573	total: 563ms	remaining: 6.68s
14:	learn: 0.3858252	total: 592ms	remaining: 6.51s
15:	learn: 0.3689252	total: 620ms	remaining: 6.35s
16:	learn: 0.3534924	total: 649ms	remaining: 6.22s
17:	learn: 0.3382094	total: 678ms	remaining: 6.1s
18:	learn: 0.3269606	total: 706ms	remaining: 5.99s
19:	learn: 0.3134611	total: 735ms	remaining

In [38]:
model.best_params_

{'depth': 10, 'iterations': 180, 'learning_rate': 0.41000000000000003}

In [39]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
df_test = pd.read_csv("../../data/test.csv")

grid = CatBoostClassifier(**model.best_params_)
model = train(grid, X_train, y_train)
submit(model, df_test, X_train)

0:	learn: 0.9743840	total: 29.4ms	remaining: 5.26s
1:	learn: 0.8760591	total: 59.3ms	remaining: 5.28s
2:	learn: 0.7919251	total: 88.7ms	remaining: 5.24s
3:	learn: 0.7314432	total: 119ms	remaining: 5.24s
4:	learn: 0.6783486	total: 149ms	remaining: 5.22s
5:	learn: 0.6336409	total: 178ms	remaining: 5.16s
6:	learn: 0.5902125	total: 205ms	remaining: 5.08s
7:	learn: 0.5580035	total: 232ms	remaining: 4.99s
8:	learn: 0.5283325	total: 260ms	remaining: 4.94s
9:	learn: 0.4944660	total: 288ms	remaining: 4.9s
10:	learn: 0.4689583	total: 315ms	remaining: 4.84s
11:	learn: 0.4424289	total: 343ms	remaining: 4.8s
12:	learn: 0.4238473	total: 370ms	remaining: 4.75s
13:	learn: 0.4051573	total: 397ms	remaining: 4.7s
14:	learn: 0.3858252	total: 424ms	remaining: 4.66s
15:	learn: 0.3689252	total: 452ms	remaining: 4.63s
16:	learn: 0.3534924	total: 479ms	remaining: 4.59s
17:	learn: 0.3382094	total: 507ms	remaining: 4.56s
18:	learn: 0.3269606	total: 536ms	remaining: 4.54s
19:	learn: 0.3134611	total: 562ms	remaini

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
df_test = pd.read_csv("../../data/test.csv")
df_test.head()
parameters = {
    "depth": list(np.arange(10, 100, 10)),
    "learning_rate": list(np.arange(0.01, 1, .1)),
    "iterations": list(np.arange(0, 300, 20))
}
grid = GridSearchCV(estimator=CatBoostClassifier(), param_grid=parameters, cv=2, n_jobs=-1)
model = train(grid, X_train, y_train)
submit(model, df_test, X_train)